In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [8]:
from fastai import * 
from fastai.vision import *

from pathlib import Path
import json 
from matplotlib import patches, patheffects
import os 
import collections
import pandas as pd

In [4]:
imgs = [] 
img_file = open('data/boxes.csv', 'r')
for line in img_file: 
    en = line.strip().split(',') 
    imgs.append( (en[0], [int(i) for i in en[1].split()] , en[2]))

In [5]:
imgs

[('img1.jpg', [121, 16, 492, 390], '0'),
 ('img2.jpg', [1057, 396, 2100, 1450], '1'),
 ('img3.jpg', [213, 39, 673, 519], '0')]

In [6]:
df = pd.DataFrame( {'fn': [ i[0] for i in imgs ] , 
                    'x0': [ i[1][0] for i in imgs],
                    'y0': [ i[1][1] for i in imgs],
                    'x1': [ i[1][2] for i in imgs],
                    'y1': [ i[1][3] for i in imgs],
                    'label': [ i[2] for i in imgs ] 
                   }, columns = ['fn', 'x0', 'y0', 'x1', 'y1', 'label'] )
df.to_csv('data/dataset.csv', index=False)

In [7]:
class ObjectList(ObjectCategoryList):
    "`ItemList` for labelled bounding boxes."
    _processor = vision.data.ObjectCategoryProcessor

    def get(self, i):
        # I imagine self.items[i] returns only one bounding box here, as the list of columns + label['x0', 'y0', 'x1', 'y1', 'label']
        bb = self.items[i]
        return ImageBBox.create(*_get_size(self.x,i), [bb[:-1]], labels=[bb[-1]], classes=self.classes, pad_idx=self.pad_idx)

data = ObjectItemList.from_csv(path='data', 
                         csv_name='dataset.csv', 
                         folder='imgs',
                         )
data = data.split_none() 
data = data.label_from_df(cols=[ 'x0', 'y0', 'x1', 'y1', 'label'], label_cls=ObjectList)
data = data.transform(None, resize_method = ResizeMethod.SQUISH, size=224, tfm_y=True)
data = data.databunch(bs=64)

TypeError: 'int' object is not iterable